In [46]:
!pip install wandb --upgrade

We import all the useful packages.

In [69]:
import sys
from pathlib import Path

IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules
if IS_KAGGLE:
    repo_path = Path("../input/microstructure-reconstruction")
elif IS_COLAB:
    from google.colab import drive

    drive.mount("/content/gdrive")
    repo_path = Path("/content/gdrive/MyDrive/microstructure-reconstruction")
else:
    repo_path = Path("/home/matias/microstructure-reconstruction")
sys.path.append(str(repo_path))

from copy import deepcopy
from importlib import reload

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics
import torchvision.models as models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
from torchvision import transforms, utils
from tqdm import tqdm

import wandb
from custom_datasets import dataset
from tools import dataframe_reformat, inspect_code, plotting, training, wandb_api

log_wandb = True

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 2, "pin_memory": True} if use_cuda else {"num_workers": 4}
print(f"[INFO]: Computation device: {device}")


[INFO]: Computation device: cpu


We initialize a wandb run, that will save our metrics

In [70]:
if log_wandb:
    import wandb

    wandb_api.login()
    run = wandb.init(
        project="microstructure-reconstruction",
        group="Naive Network",
        job_type="train",
    )


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/matias/.netrc


Parameters of our run:

In [71]:
if log_wandb:
    config = wandb.config
else:
    config = {}

config["job_type"] = run.job_type if "run" in locals() else "test"
config["log_wandb"] = log_wandb


In [72]:
scaler, train_df, test_df = wandb_api.fetch_train_test_df(repo_path, normalized=True)
mean = train_df.iloc[:, 1:-1].mean().to_numpy()
predictions = torch.FloatTensor(np.tile(mean, (len(test_df), 1)))
targets = torch.FloatTensor(test_df.iloc[:, 1:-1].to_numpy())


In [73]:
for i in range(50):
    wandb.log(
        {
            "val_loss": torchmetrics.MeanAbsoluteError()(predictions, targets), 
            "train_loss": torchmetrics.MeanAbsoluteError()(predictions, targets),
            "mae": torchmetrics.MeanAbsoluteError()(predictions, targets),
            "mape": torchmetrics.MeanAbsolutePercentageError()(predictions, targets),
            "smape": torchmetrics.SymmetricMeanAbsolutePercentageError()(
                predictions, targets
            ),
            "r2_score": torchmetrics.R2Score(num_outputs=23)(predictions, targets),
            "cosine_similarity": torchmetrics.CosineSimilarity(reduction="mean")(
                predictions, targets
            ),
            "epoch": i,
        }
    )
run.finish()

cosine_similarity,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
mae,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mape,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
r2_score,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
smape,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
cosine_similarity,0.90602
epoch,49
mae,0.1638


In [68]:
torchmetrics.MeanAbsoluteError()(predictions, targets)
torchmetrics.MeanAbsolutePercentageError()(predictions, targets)
torchmetrics.CosineSimilarity(reduction="mean")(predictions, targets)
torchmetrics.R2Score(num_outputs=23)(predictions, targets)


tensor(-0.1041)